# Artificial Neural Networks

In [2]:
import numpy as np
seed = 42
rng = np.random.default_rng(seed=seed)

## Network Class

We shall start writing the `Network` class. The two methods that are indispensable for any ML class are :
- `fit`
- `predict`

Fitting a neural network model requires us to compute two passes on the data :
- `forward`
- `backward`

We need to start at some place by initializing the network and various hyperparameters and this requires an `init` method :
- `init`

In most of these methods, we would have to take the help of certain helper functions :
- `activations`
- `losses`

This is the process. But we will work through it in the reverse order so that each step of the process does not have any forward references :
`helpers -> init -> forward -> backward -> fit -> predict`

The skeleton of the class is given in the code block that follows. For ease of exposition, we are going to discuss the methods on at a time and then plugh them into the class right at the end.

In [3]:
class Network :
    
    def __init__(self, layers, activation_choice="relu", output_choice="softmax", loss_choice="cce") :
        pass
    
    def forward(self, X) :
        pass
    
    def backward(self, Y, Y_hat) :
        pass
    
    def fit(self, X, Y, lr=0.01, epochs=100, batch_size=100) :
        pass
    
    def predict(self, X) :
        pass

## Activation Functions

### Hidden Layer

We will look at 2 functions for the hidden layers. Both of these functions will be **applied element-wise**. The input to these functions can be scalars, vectors or matrices

- Sigmoid :
$$
    g(z) = \frac {1} {1 + e^{-z}}
$$

Its derivative :

$$
    g'(z) = g(z)(1 - g(z))
$$

- ReLU ( Rectified Linear Unit ) :
$$
    g(x)=\begin{cases}
    z, & z \ge 0 \\ 
    0, & z<0
    \end{cases}
$$

Its derivative :

$$
    g'(x)=\begin{cases}
    1, & z \ge 0 \\ 
    0, & z<0
    \end{cases}
$$


In [4]:
def sigmoid(z) :
    return 1 / (1 + np.exp(-z))

def grad_sigmoid(z) :
    return sigmoid(z) * (1 - sigmoid(z))

def relu(z) :
    return np.where(z >= 0, z, 0)

def grad_relu(z) :
    return np.where(z >= 0, 1, 0)


# A dictionary of activation functions will be used while initializing the network
hidden_act = {"sigmoid": sigmoid, "relu": relu}
grad_hiddent_act = {"sigmoid": grad_sigmoid, "relu": grad_relu}

## Output Layer

We will look at 2 activation functions for the output layer :
    
- Identity ( For regression )
$$
    g(z) = z
$$

- Softmax ( For classification ) :
The input to the softmax function will always be a matrix of size $n \times k$. Since we need a probability distribution for each data point, **the softmax will be computed row-wise**

$$
    g(\textbf Z) = 
    \begin{pmatrix}
    ... & ... & ... \\
    ... & \frac {e^{Z_{ij}}} {\sum \limits_{j=1}^{k} e^{Z_{ij}}} & ... \\
    ... & ... & ... \\
    \end{pmatrix}
$$

**To avoid overflow, we will subtract the row-wise maximum from each row while computing the softmax**

In [5]:
def identity(z) :
    return z

def softmax(z) :
    """
    Row-wise softmax
    """
    # Check if z is a matrix
    assert z.ndim == 2
    
    # To prevent overflow, subtract row-wise maximum
    z -= z.max(acis=1, keepdims=True)
    
    # Compute row-wise softmax
    prob = np.exp(z) / np.exp(z).sum(axis=1, keepdims=True)
    
    # Check if each row is a probability distribution
    assert np.allclose(prob.sum(axis=1), np.ones(z.shape[0]))
    
    return prob

output_act = {"softmax": softmax, "identity": identity}

## Loss

We will use 2 types of losses :

- Least Square Loss ( For Regression ) :

$$
    L(\hat {\textbf y}, \textbf y) = \frac {1} {2} (\hat {\textbf y} - \textbf y)^{T}(\hat {\textbf y} - \textbf y)
$$

- Categorial Cross-Entropy Loss ( For Classification ) :
    
$$
    L(\hat {\textbf Y}, \textbf Y) = - \textbf 1_{n}^T(\textbf Y \odot log(\hat {\textbf Y})\textbf1-{k}
$$

In our implementation, we will assume that the arguments to the loss function are always matrices of size $n \times k$. In the case of regression, $k = 1$.

In [ ]:
def lease_square(y, y_hat) :
    return 0.5 * np.sum((y_hat - y) * (y_hat - y))

def cce(Y, Y_hat) :
    return -np.sum(Y * np.log(Y_hat))

losses = {"least_squares": least_squares, "cce": cce}

## Initialization

Here, we will look at two parts :

### Network architecture

The following components mainly determine the structure of the network :
- Number of layers
- Number of neuron per layer
We will use $l$ to index the layers. The network has $L$ layers in all.
- $l = 0$ : Input layer
- $1 \le l \le L - 1$ : Hidden layers
- $l = L$ : Output layer

We shall represent the number of layers and neurons using a list `layers`. The variable $L$ will never make an explicit appearance anywhere, instead will use `range(len(layers))` to iterate through the layers.

| Layer           | Number of neurons |
| -----------     | -----------       |
| Input layer     | `layers[0]`       |
| Hidden layer 1  | `layers[1]`       |
| Hidden layer 2  | `layers[2]`       |
| Hidden layer 3  | `layers[3]`       |
| **.......**     | `layers[...]`     |
| Output layer    | `layers[-1]`      |

One useful task is to compute the total number of parameters in each network. This will come handy later on.

In [9]:
def count_params(layers) :
    num_params = 0
    for l in range(1, len(layers)) :
        num_weights = layers[l - 1] * layers[l]
        num_biases = layers[l]
        num_params += (num_weights + num_biases)
    return num_params

### Parameter initialization

- The weight matrix at layer $l$ has a size of `layers[l - 1] x layers[l]`
- The bias at layer $l$ is a vector of size `layers[l]`
- We will store all these weights in a list `w` of the same size as `layers`. So, `W[l]` will correspond to $\textbf W_l$. Since there are $L$ weight matrices `W[0]` would be set to `None`. Recall that size of the list if $L + 1$
- A similar list would be required for `b`

To make the gradient descent update simpler, it will be useful to have a **master vector**, $\textbf \theta$, that has a refernce to all the parameters in the network. We will do the same for the gradients $\textbf \theta^{(g)}$. So, whenever $\textbf \theta$ is updated, the weights $\textbf W_l$ will also be updated and vice-versa